In [ ]:
from joltml import Experiment, Xgboost, Sklearn
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
# Logging is done implicitly
dataset = fetch_california_housing(as_frame=True)["frame"]
dataset = dataset.dropna()
training_set = dataset[:16000]
prediction_set = dataset.iloc[16000:,:-1]


In [ ]:
import pandas as pd
experiment = Experiment(training_set,experiment_id='trial1')
experiment.add_models([Xgboost()])
experiment.regression(target_names=['MedHouseVal'],splits=0.2)
# Write results and default evaluation metrics
y = experiment.predict(prediction_set)



In [ ]:
params = {
    'booster': {'type': 'categorical', 'values': ['gbtree', 'gblinear', 'dart']},
    'lambda': {'type': 'float', 'minimum': 1e-8, 'maximum': 1.0},
    'alpha': {'type': 'float', 'minimum': 1e-8, 'maximum': 1.0},
}

experiment = Experiment(training_set, experiment_id='trial1')
experiment.add_models([Xgboost()])
experiment.regression_optimize(
    targets=['MedHouseVal'], splits=0.2, n_trials=10, params=params)
# Write results and default evaluation metrics
y = experiment.predict(prediction_set)


In [ ]:
experiment = Experiment(training_set)
y = experiment.add_models([Xgboost()]).regression(targets=['MedHouseVal'], splits=[0.8,0.2]).predict(prediction_set)

In [ ]:
from sklearn.linear_model import ElasticNet
experiment = Experiment(training_set)
y = experiment.add_models([Sklearn(ElasticNet())]).regression(targets=['MedHouseVal'],splits=[0.8,0.2]).predict(prediction_set)

In [ ]:
from sklearn.linear_model import ElasticNet, LinearRegression, Lasso

experiment = Experiment(training_set, experiment_id="trial1")
y = experiment.add_models([Xgboost(),
                           Sklearn(ElasticNet()),
                           Sklearn(LinearRegression()),
                           Sklearn(Lasso()),
                           ]).regression(targets=['MedHouseVal'],splits=[0.8, 0.2]).predict(prediction_set)


In [ ]:
from sklearn.linear_model import ElasticNet,\
LinearRegression, Ridge, RidgeCV, SGDRegressor, Lars, Lasso,\
LassoLars, ARDRegression, HuberRegressor, QuantileRegressor,\
RANSACRegressor, TheilSenRegressor

experiment = Experiment(training_set, experiment_id="trial1")
y = experiment.add_models([Xgboost(),
                           Sklearn(ElasticNet()),
                           Sklearn(LinearRegression()),
                           Sklearn(Ridge()),
                           Sklearn(RidgeCV()),
                           Sklearn(SGDRegressor()),
                           Sklearn(Lars()),
                           Sklearn(Lasso()),
                           Sklearn(LassoLars()),
                           Sklearn(ARDRegression()),
                           Sklearn(HuberRegressor()),
                           Sklearn(QuantileRegressor()),
                           Sklearn(RANSACRegressor()),
                           Sklearn(TheilSenRegressor())]).regression_on(
    targets=['MedHouseVal']).fit(splits=0.2).predict(prediction_set)


In [ ]:
from joltml import Experiment, Pytorch
import torch
import torch.nn as nn
import pandas as pd

a = 1
b = 0.5

X = torch.linspace(0,10,1000)
y = a * X + b

data = pd.DataFrame(zip(X,y),columns=['X','y'])
training_set = data.iloc[:800]
prediction_set = data.iloc[800:,:-1]

class RegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.a = nn.Parameter(torch.randn(
            1, dtype=torch.float), requires_grad=True)
        self.b = nn.Parameter(torch.randn(
            1, dtype=torch.float), requires_grad=True)

    # <- "x" is the input data (e.g. training/testing features)
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.a*x + self.b


experiment = Experiment(training_set)
y = experiment.add_models([Pytorch(RegressionModel())]).regression(
    targets=['y'], splits=[0.8, 0.2]).predict(prediction_set)


In [ ]:
from joltml import Experiment, Pytorch
import torch.nn as nn
import numpy as np
from sklearn.datasets import fetch_california_housing
# Logging is done implicitly
dataset = fetch_california_housing(as_frame=True)["frame"]
dataset = dataset.dropna()
dataset.astype(np.float32)
dataset = dataset.sample(frac=1)
training_size = int(0.9*len(dataset))
training_set = dataset[:training_size]
prediction_set = dataset.iloc[training_size:,:-1]
prediction_y = dataset.iloc[training_size:,-1]

model = nn.Sequential(
    nn.Linear(training_set.shape[1]-1, 24),
    nn.ReLU(),
    nn.Linear(24, 12),
    nn.ReLU(),
    nn.Linear(12, 6),
    nn.ReLU(),
    nn.Linear(6, 1)
)

experiment = Experiment(training_set)
y = experiment.add_models([Pytorch(model)]).regression(
    targets=['MedHouseVal'], splits=[0.8, 0.2]).predict(prediction_set)
from joltml.joltmeter import RegressionMetrics
print(RegressionMetrics.mean_absolute_error.evaluate(y[0].detach().numpy(),prediction_y))